In [1]:
# 모듈 임포트
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import joblib
import time

In [2]:
# 파생 피처 생성 함수
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

In [3]:
# 시퀀스 생성 함수
def create_sequences(data, labels, seq_len=8):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(labels[i+seq_len - 1])
    return np.array(X), np.array(y)


In [4]:
# 테스트셋 로드 및 전처리
test_df = pd.read_csv('../realtime/test/9대0.1.csv')  # 경로 확인
test_df = add_features(test_df).dropna()

features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']

X_raw = test_df[features].values
y_raw = test_df['label'].values.astype(int)

scaler = joblib.load('./model/scaler_cnn.joblib')
X_scaled = scaler.transform(X_raw)

# CNN 입력 시퀀스 생성
seq_len = 8
X_seq, y_seq = create_sequences(X_scaled, y_raw, seq_len)
X_seq = X_seq.reshape((X_seq.shape[0], seq_len, len(features)))

In [5]:
# 모델 로드
model = tf.keras.models.load_model('./model/cnn1d_model.h5')

# 예측
y_pred_prob = model.predict(X_seq)
y_pred = (y_pred_prob.flatten() > 0.3).astype(int)

845/845 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [6]:
# 후처리
def postprocess_anomalies(y_pred_bin, min_consecutive=8):
    y_post = np.zeros_like(y_pred_bin)
    count = 0
    for i, val in enumerate(y_pred_bin):
        if val == 1:
            count += 1
        else:
            if count >= min_consecutive:
                y_post[i - count:i] = 1
            count = 0
    if count >= min_consecutive:
        y_post[len(y_pred_bin)-count:] = 1
    return y_post

y_pred_post = postprocess_anomalies(y_pred, min_consecutive=1)
y_true_post = postprocess_anomalies(y_seq, min_consecutive=1)


In [7]:
# 평가 지표 출력
print("📋 후처리 적용 후 CNN 분류 리포트:")
print("📋 후처리 적용 후 CNN 분류 리포트:")
print(classification_report(
    y_true_post,
    y_pred_post,
    labels=[0, 1],
    target_names=['정상', '아크'],
    zero_division=0  # 예: precision 0/0 방지
))


📋 후처리 적용 후 CNN 분류 리포트:
📋 후처리 적용 후 CNN 분류 리포트:
              precision    recall  f1-score   support

          정상       1.00      0.91      0.95     26992
          아크       0.00      0.13      0.00        30

    accuracy                           0.91     27022
   macro avg       0.50      0.52      0.48     27022
weighted avg       1.00      0.91      0.95     27022

